In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
from pyreadstat import read_sas7bdat

# Para rodar as regressões de painel
from linearmodels import PanelOLS
from linearmodels.panel import PanelOLS, RandomEffects, PooledOLS
from linearmodels.panel.model import PanelOLS, FamaMacbeth, RandomEffects, PooledOLS

# Para a winsorização
from scipy.stats import mstats

# Configurações de exibição do Pandas para mostrar todas as colunas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# --- Importar a base de dados ---
# Importar base formato .sas7bdat
# Substitua o caminho do arquivo pelo seu
df, meta = read_sas7bdat("/Users/macvini/Library/CloudStorage/OneDrive-Pessoal/Mestrado/base_final_mestrado.sas7bdat", encoding='utf-8')

# Excluir linhas com missing ou valor vazio para 'cliente'
df = df.dropna(subset=['cliente'])
df = df[df['cliente'] != '']
# Criar ID único para o cliente (equivalente ao `egen group` do Stata)
df['id_cliente'] = df.groupby('cliente').ngroup()

# --- Engenharia de Variáveis ---

# Converter a variável de data para um formato datetime do Python
# A data no Stata (DT_NASCIMENTO) é o número de dias desde 01/01/1960.
# O Stata também usa `mdy(1,1,1960) + DT_NASCIMENTO`, o que é estranho. A conversão mais precisa seria:
df['DT_NASCIMENTO'] = pd.to_datetime('1960-01-01') + pd.to_timedelta(df['DT_NASCIMENTO'], unit='D')
# Criar variável idade a partir da data de nascimento
hoje = pd.to_datetime('today')
df['idade'] = (hoje - df['DT_NASCIMENTO']).dt.days / 365.25
df['idade_int'] = df['idade'].astype(int)

# Unificar investimentos em uma única variável
colunas_invest_ext = ['INVEST_EXT_RENDA_VARIAVEL', 'INVEST_NO_EXTERIOR', 'INVEST_EXTERIOR', 'INVEST_EXT_RENDA_FIXA']
df['investimento_exterior'] = df[colunas_invest_ext].sum(axis=1, skipna=True)
# O comando `rowtotal` do Stata soma apenas valores não nulos. O `sum(axis=1, skipna=True)` faz o mesmo.

# Variável dummy para sexo
df['sexo_dummy'] = df['SEXO'].map({'M': 1, 'F': 0}).astype(pd.Int64Dtype()) # Usa Int64Dtype para aceitar NaN

# Dummies de Região
regioes = {
    'Norte': ["AC", "AP", "AM", "PA", "RO", "RR", "TO"],
    'Nordeste': ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"],
    'Sudeste': ["ES", "MG", "RJ", "SP"],
    'Sul': ["PR", "RS", "SC"],
    'Centro-Oeste': ["DF", "GO", "MT", "MS"]
}
for regiao, ufs in regioes.items():
    df[f'regiao_{regiao.lower()}'] = df['UF_CADASTRO'].isin(ufs).astype(int)

# Dummy para 'Estilo Investidor'
df['estilo_investidor'] = (df['NM_TIP_CTRA'] == "ESTILO INVESTIDOR").astype(int)

# Dummies para Estado Civil
ec_map = {
    1: 'solteiro',
    2: 'casado', 3: 'casado', 4: 'casado', 8: 'casado', 9: 'casado', 11: 'casado', 12: 'casado',
    6: 'separado', 7: 'separado',
    5: 'viuvo'
}
df['estado_civil_grupo'] = df['EST_CIVIL'].map(ec_map).fillna('nao_informado')
df = pd.get_dummies(df, columns=['estado_civil_grupo'], prefix='ec', dtype=int)

# Dummies para Escolaridade
esc_map = {
    1: 'baixa', 2: 'baixa',
    3: 'media', 4: 'media', 9: 'media',
    5: 'alta', 6: 'alta', 7: 'alta', 8: 'alta',
    0: 'missing'
}
df['escolaridade_grupo'] = df['ESCOLAR'].map(esc_map).fillna('nao_informado')
df = pd.get_dummies(df, columns=['escolaridade_grupo'], prefix='esc', dtype=int)

# Dummies para Perfil Investidor
df['prfl_codigo'] = df['CD_PRFL_API'].replace(0, 5)
df['prfl_codigo'] = df['prfl_codigo'].fillna(5).astype(int)
perfil_map = {
    1: 'conservador', 2: 'moderado', 3: 'arrojado', 4: 'agressivo', 5: 'nao_resp'
}
df['perfil_grupo'] = df['prfl_codigo'].map(perfil_map)
df = pd.get_dummies(df, columns=['perfil_grupo'], prefix='perfil', dtype=int)

# Dummies para Ocupação usando expressões regulares
ocup_map = {
    'Administração': r'ADMINISTRADOR|CONTADOR|ANALISTA|CONSULTOR|ECONOMISTA',
    'Servidor Público': r'SERVIDOR PUBLICO|DEPUTADO|PREFEITO|SECRETARIO|MAGISTRADO|PROCURADOR',
    'Saúde': r'MEDICO|ENFERMEIRO|FISIOTERAPEUTA|ODONTOLOGO|FARMACEUTICO|NUTRICIONISTA|FONOAUDIOLOGO|PSICOLOGO|TERAPEUTA',
    'Educação': r'PROFESSOR|ESTUDANTE|ESTAGIARIO|BOLSISTA|PEDAGOGO',
    'Autônomo/Comércio': r'COMERCIANTE|AMBULANTE|TAXISTA|VENDEDOR|FEIRANTE|REPRESENTANTE COMERCIAL',
    'Agropecuária': r'AGRICULTOR|PECUARISTA|PESCADOR|AVICULTOR|RURAL|FLORICULTOR|AGRONOMO|AGROPECUARISTA',
    'Industrial': r'MECANICO|ELETRICISTA|OPERADOR|CONSTRUCAO|MARCENEIRO|INDUSTRIARIO|SERRALHEIRO|TECNIC',
    'Justiça': r'ADVOGADO|DELEGADO|DEFENSOR|PROMOTOR|JUIZ|OFICIAL DE JUSTICA|TABELIAO|CARTORIO',
    'Segurança': r'POLICIAL|MILITAR|VIGILANTE|SEGURANCA|BOMBEIRO',
    'Cultura/Comunicação': r'MUSICO|ATOR|ARTESAO|JORNALISTA|ESCULTOR|PUBLICITARIO|FOTOGRAFO|LOCUTOR'
}

df['grupo_ocupacao'] = 'Outros'
for grupo, regex in ocup_map.items():
    df.loc[df['DS_OCUPACAO'].str.contains(regex, case=False, na=False), 'grupo_ocupacao'] = grupo
df = pd.get_dummies(df, columns=['grupo_ocupacao'], prefix='oc', dtype=int)

# Excluir variáveis auxiliares
vars_to_drop = [
    'DT_NASCIMENTO', 'SEXO', 'cliente', 'idade', 'UF_CADASTRO', 'CARTEIRA', 'CD_TIP_CTRA', 
    'NM_TIP_CTRA', 'EST_CIVIL', 'ESCOLAR', 'CD_PRFL_API', 'TX_DCR_PRFL', 'prfl_codigo', 
    'GRUPO_OCUPACAO', 'CD_NTZ_OCUPACAO', 'DS_NTZ_OCUPACAO', 'CD_OCUPACAO', 
    'DS_OCUPACAO', 'INVEST_EXT_RENDA_VARIAVEL', 'INVEST_NO_EXTERIOR', 
    'INVEST_EXTERIOR', 'INVEST_EXT_RENDA_FIXA'
]
df = df.drop(columns=[c for c in vars_to_drop if c in df.columns], errors='ignore')

# Variáveis com dados regionais
dados_regionais = {
    'regiao_norte': {'escolaridade': 9.2, 'renda': 2421.7, 'idh': 0.6847, 'pib': 33123},
    'regiao_nordeste': {'escolaridade': 8.3, 'renda': 2078.0, 'idh': 0.6487, 'pib': 25401},
    'regiao_sudeste': {'escolaridade': 10.0, 'renda': 3514.0, 'idh': 0.7537, 'pib': 63327},
    'regiao_sul': {'escolaridade': 10.1, 'renda': 3423.7, 'idh': 0.7563, 'pib': 55942},
    'regiao_centro-oeste': {'escolaridade': 10.1, 'renda': 3604.0, 'idh': 0.7533, 'pib': 65651},
}

for var in ['escolaridade_regiao', 'renda_regional', 'idh_regional', 'pib_percapita_regional']:
    df[var] = np.nan

for regiao, valores in dados_regionais.items():
    mask = df[f'regiao_{regiao}'] == 1
    df.loc[mask, 'escolaridade_regiao'] = valores['escolaridade']
    df.loc[mask, 'renda_regional'] = valores['renda']
    df.loc[mask, 'idh_regional'] = valores['idh']
    df.loc[mask, 'pib_percapita_regional'] = valores['pib']

# --- Variáveis Dependentes ---

# Variável de diversificação
df['soma_complex'] = df[['MULTIMERCADOS', 'RENDA_VARIAVEL', 'INVEST_ALTERNATIVOS', 'investimento_exterior']].sum(axis=1)
df['soma_total'] = df[['RENDA_FIXA_POS_CDI', 'RENDA_FIXA_PRE', 'RENDA_FIXA_INFLACAO', 'MULTIMERCADOS', 'RENDA_VARIAVEL', 'INVEST_ALTERNATIVOS', 'investimento_exterior']].sum(axis=1)
df['diver'] = df['soma_complex'] / df['soma_total']
df['diver'] = df['diver'].fillna(0)
# 'Diver' não pode ser maior que 1, então vamos garantir isso
df.loc[df['diver'] > 1, 'diver'] = 1

# Variável de complexidade financeira
df['complex'] = (df['soma_complex'] > 0).astype(int)

# --- Análises de Painel (Preparação) ---

# Converter para painel: id_cliente e anomes
# 'anomes' precisa ser um tipo de data para o PanelOLS
df['anomes'] = pd.to_datetime(df['anomes'], format='%Y%m')
df = df.set_index(['id_cliente', 'anomes'])
df = df.sort_index()

# Remover duplicatas (já feito com o `set_index` se `id_cliente` e `anomes` forem únicos para cada observação)
df = df.loc[~df.index.duplicated(keep='first')]

# Criar variáveis de variação e skewness (equivalente ao `xtset` do Stata)
df['log_renda'] = np.log(df['renda'] + 1)
df['delta_y'] = df.groupby(level='id_cliente')['log_renda'].diff()

# Código para calcular o skewness (complexo, mas direto no pandas)
df['ano'] = df.index.get_level_values('anomes').year
df['regiao_codigo'] = df[['regiao_norte', 'regiao_nordeste', 'regiao_sudeste', 'regiao_sul', 'regiao_centro-oeste']].dot(
    pd.Series([1, 2, 3, 4, 5], index=['regiao_norte', 'regiao_nordeste', 'regiao_sudeste', 'regiao_sul', 'regiao_centro-oeste'])
)
df['delta_y_anual'] = df.groupby(['id_cliente', 'ano'])['delta_y'].transform('mean')
df['media_dy'] = df.groupby(['regiao_codigo', 'ano'])['delta_y_anual'].transform('mean')
df['sd_dy'] = df.groupby(['regiao_codigo', 'ano'])['delta_y_anual'].transform('std')
df['delta_y_pad'] = (df['delta_y_anual'] - df['media_dy']) / df['sd_dy']
df['skew_aux'] = df['delta_y_pad']**3
df['skew'] = df.groupby(['regiao_codigo', 'ano'])['skew_aux'].transform('mean')
df['grupo_n'] = df.groupby(['regiao_codigo', 'ano'])['skew_aux'].transform('count')
df.loc[df['grupo_n'] < 30, 'skew'] = np.nan
df['skew_final'] = df.loc[df['ano'] > 2021, 'skew']
df['skew_media_regional'] = df.groupby('regiao_codigo')['skew_final'].transform('mean')
df['skew_proxy'] = df['skew'].fillna(df['skew_media_regional'])

# Normalização com log natural (LN)
df['ln_diver'] = np.log(df['diver'] + 0.01)
# O `np.log` já lida com os valores zero ou missing corretamente

df['ln_renda'] = np.log(df['renda_regional'].fillna(1))
df['ln_ESC'] = np.log(df['escolaridade_regiao'].fillna(1))
df['ln_IDH'] = np.log(df['idh_regional'].fillna(1))
df['ln_PIB'] = np.log(df['pib_percapita_regional'].fillna(1))

# Winsorização para controlar outliers
# `mstats.winsorize` é a melhor tradução
for var in ['ln_diver', 'ln_renda', 'ln_ESC', 'ln_IDH', 'ln_PIB']:
    if var in df.columns:
        df[f'{var}_w'] = mstats.winsorize(df[var], limits=(0.01, 0.01))

# --- Início das Análises com Statsmodels e Linearmodels ---

# Matriz de Correlação
corr_vars = ['ln_diver_w', 'ln_renda_w', 'ln_ESC_w', 'ln_IDH_w', 'ln_PIB_w', 'idade_int', 'sexo_dummy', 'skew_proxy']
corr_matrix = df[corr_vars].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlação')
plt.show()

# Modelos com Pooled, FE e RE (usando a biblioteca `linearmodels`)
# A biblioteca `linearmodels` é a melhor para regressões de painel em Python.
# Lembre-se de definir as variáveis como "entidades" e "tempo" no `PanelOLS`.

# Fórmula básica para os modelos
formula = 'ln_diver_w ~ ln_renda_w + ln_IDH_w + sexo_dummy + idade_int'

# Modelo Pooled
mod_pooled = PooledOLS.from_formula(formula, data=df)
res_pooled = mod_pooled.fit(cov_type='robust')
print("\nModelo Pooled:")
print(res_pooled)

# Modelo de Efeitos Fixos
mod_fe = PanelOLS.from_formula(formula + ' + EntityEffects', data=df)
res_fe = mod_fe.fit(cov_type='clustered', cluster_entity=True)
print("\nModelo de Efeitos Fixos:")
print(res_fe)

# Modelo de Efeitos Aleatórios
mod_re = RandomEffects.from_formula(formula, data=df)
res_re = mod_re.fit(cov_type='clustered', cluster_entity=True)
print("\nModelo de Efeitos Aleatórios:")
print(res_re)

# Teste de Hausman (requer a biblioteca `hausman`, que é mais complexa de usar diretamente)
# O mais simples é usar os resultados do `linearmodels` para tomar a decisão: se `fe` e `re` dão resultados muito diferentes, use `fe`.
# O `linearmodels` tem um teste de Hausman integrado, mas a sintaxe pode ser um pouco diferente.
# Você pode fazer o teste manualmente comparando os coeficientes:
# hausman_stat = (res_fe.params - res_re.params).T @ np.linalg.inv(res_re.cov - res_fe.cov) @ (res_fe.params - res_re.params)

# --- Regressões para as Hipóteses ---
# H1 e H1a — Efeito da complexidade e contexto regional
formula_h1 = 'ln_diver_w ~ complex + ln_ESC_w + ln_renda_w + regiao_norte + regiao_nordeste + regiao_sul + regiao_centro_oeste + sexo_dummy + idade_int + EntityEffects'
mod_h1 = PanelOLS.from_formula(formula_h1, data=df)
res_h1 = mod_h1.fit(cov_type='clustered', cluster_entity=True)
print("\nModelo H1 e H1a:")
print(res_h1)

# H2 — Efeito do risco cíclico de renda
formula_h2 = 'ln_diver_w ~ skew_proxy + ln_ESC_w + ln_renda_w + regiao_norte + regiao_nordeste + regiao_sul + regiao_centro_oeste + sexo_dummy + idade_int + EntityEffects'
mod_h2 = PanelOLS.from_formula(formula_h2, data=df)
res_h2 = mod_h2.fit(cov_type='clustered', cluster_entity=True)
print("\nModelo H2:")
print(res_h2)

# H3 — Efeito do IDH em portfólios de alta renda
df_alta_renda = df[df['renda'] > 20000]
formula_h3 = 'ln_diver_w ~ ln_IDH_w + ln_renda_w + ln_ESC_w + regiao_norte + regiao_nordeste + regiao_sul + regiao_centro_oeste + sexo_dummy + idade_int + EntityEffects'
mod_h3 = PanelOLS.from_formula(formula_h3, data=df_alta_renda)
res_h3 = mod_h3.fit(cov_type='clustered', cluster_entity=True)
print("\nModelo H3:")
print(res_h3)

# --- Estatísticas Descritivas ---
# O método `describe()` é o mais comum para estatísticas descritivas
print("\nEstatísticas Descritivas:")
print(df[['diver', 'ln_diver_w', 'ln_renda_w', 'ln_ESC_w', 'ln_IDH_w', 'idade_int', 'sexo_dummy', 'complex', 'skew_proxy', 'perfil_conservador', 'perfil_moderado', 'perfil_arrojado']].describe())

# --- Visualização de Dados (o poder do Python) ---
# Gráfico de dispersão entre a diversificação e a renda, com linha de tendência
sns.regplot(x='ln_renda_w', y='ln_diver_w', data=df)
plt.title('Diversificação vs. Renda')
plt.xlabel('Log da Renda (Winsorizado)')
plt.ylabel('Log da Diversificação (Winsorizado)')
plt.show()

# Boxplot da diversificação por grupo de ocupação
plt.figure(figsize=(12, 6))
sns.boxplot(x='grupo_ocupacao', y='diver', data=df)
plt.title('Diversificação por Grupo de Ocupação')
plt.xticks(rotation=45)
plt.show()

# Boxplot da diversificação por perfil de investidor
plt.figure(figsize=(8, 6))
sns.boxplot(x='perfil_grupo', y='diver', data=df)
plt.title('Diversificação por Perfil do Investidor')
plt.show()

print("\n--- Análises Finais ---")
# O resto do seu script pode ser traduzido de forma similar. 
# Basta usar `PanelOLS` com os termos adicionais na fórmula.

ModuleNotFoundError: No module named 'linearmodels'